In [1]:
import pandas as pd
import os
import shutil
import sys

In [2]:
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\aakas\Documents\Projects\Thyroid-disease-detection\research


In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aakas\\Documents\\Projects\\Thyroid-disease-detection'

['thyroid_20230819_200132.csv', 'thyroid_20230819_200614.csv']

#### Column Check


In [7]:
import yaml
from thyroid.utils.exception import customException

In [5]:
def read_yaml_file(file_path: str) -> dict:
    try:
        with open(file_path, "rb") as yaml_file:
            return yaml.safe_load(yaml_file)
    except Exception as e:
        raise customException(e, sys) from e

In [27]:
yaml_file_path = os.path.join("config", "config.yaml")
config_data = read_yaml_file(yaml_file_path)
expected_column_length = config_data['train_column_length']


In [9]:
expected_column_length

30

In [10]:
artifact_directory = "artifact"
ingested_directory = "ingested_data"
bad_directory = "bad_data"

In [11]:
ingested_data_directory = os.path.join(artifact_directory, ingested_directory)

# Create the ingested_data directory inside the artifact directory if it doesn't exist
bad_data_directory = os.path.join(artifact_directory, bad_directory)
if not os.path.exists(bad_data_directory):
    os.makedirs(bad_data_directory)


In [12]:
ingested_data_directory

'artifact\\ingested_data'

In [18]:
df = pd.read_csv("artifact\\ingested_data\\thyroid_20230825_144907.csv", header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,29,F,f,f,f,f,f,f,f,t,...,f,?,f,?,f,?,f,?,other,negative
1,29,F,f,f,f,f,f,f,f,f,...,t,128,f,?,f,?,f,?,other,negative
2,41,F,f,f,f,f,f,f,f,f,...,f,?,f,?,f,?,t,11,other,negative
3,36,F,f,f,f,f,f,f,f,f,...,f,?,f,?,f,?,t,26,other,negative
4,32,F,f,f,f,f,f,f,f,f,...,f,?,f,?,f,?,t,36,other,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,t,64,t,0.83,t,77,f,?,SVI,negative
9168,22,M,f,f,f,f,f,f,f,f,...,t,91,t,0.92,t,99,f,?,SVI,negative
9169,69,M,f,f,f,f,f,f,f,f,...,t,113,t,1.27,t,89,f,?,SVI,I
9170,47,F,f,f,f,f,f,f,f,f,...,t,75,t,0.85,t,88,f,?,other,negative


In [17]:
df = pd.read_csv("artifact\\ingested_data\\thyroid_20230825_144724.csv", header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,negative
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,negative
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,negative
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,70,M,f,f,f,f,f,f,f,f,...,t,155,t,1.05,t,148,f,?,SVI,negative
2796,73,M,f,t,f,f,f,f,f,f,...,t,63,t,0.88,t,72,f,?,other,negative
2797,75,M,f,f,f,f,f,f,f,f,...,t,147,t,0.8,t,183,f,?,other,negative
2798,60,F,f,f,f,f,f,f,f,f,...,t,100,t,0.83,t,121,f,?,other,negative


In [19]:
def check_number_of_columns(df,expected_column_length):
    if df.shape[1] == expected_column_length:
        return True
    else:
        return False

In [20]:
def number_of_complete_missing_data_columns(df):
    # Identify columns with all '?'
    columns_with_all_question_marks = []
    for column in df.columns:
        if all(df[column] == '?'):
            columns_with_all_question_marks.append(column)

    # Print the column names with all '?'
    print("Columns with all '?' values:", columns_with_all_question_marks)
    return(len(columns_with_all_question_marks))

In [21]:
# data source validation checks
for filename in os.listdir(ingested_data_directory):
    if filename.endswith(".csv"):
        file = os.path.join(ingested_data_directory, filename)
        df = pd.read_csv(file)
        
        # coulmns check
        if check_number_of_columns(df,expected_column_length):
            print("true")
        else:
            print("bad column length - 30 columns for train needed")
            print("moving file {} to bad data directory".format(file))
            shutil.move(file, bad_data_directory)

        # check for missing data
        missing_value = number_of_complete_missing_data_columns(df)
        print(missing_value)
        if missing_value <= 2:
            print("pass")
        else:
            print("move to bad directory")



        




true
Columns with all '?' values: ['?']
1
pass
true
Columns with all '?' values: []
0
pass
bad column length - 30 columns for train needed
moving file artifact\ingested_data\thyroid_20230825_145024.csv to bad data directory
Columns with all '?' values: []
0
pass


In [40]:
# data set merging

files = os.listdir(ingested_data_directory)
files

['thyroid_20230825_144724.csv', 'thyroid_20230825_144907.csv']

In [44]:
# Load each dataset and assign column names
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(ingested_data_directory, file), header=None)
    dfs.append(df)

In [53]:
column = config_data['columns']
column_names = [list(column.keys())[0] for column in column]
column_names

['age',
 'sex',
 'on_thyroxine',
 'query_on_thyroxine',
 'on_antithyroid_medication',
 'sick',
 'pregnant',
 'thyroid_surgery',
 'I131_treatment',
 'query_hypothyroid',
 'query_hyperthyroid',
 'lithium',
 'goitre',
 'tumor',
 'hypopituitary',
 'psych',
 'TSH_measured',
 'TSH',
 'T3_measured',
 'T3',
 'TT4_measured',
 'TT4',
 'T4U_measured',
 'T4U',
 'FTI_measured',
 'FTI',
 'TBG_measured',
 'TBG',
 'referral_source',
 'class']

In [54]:
dfs[0].columns = column_names

In [58]:
# Assign column names to each DataFrame
for df in dfs:
    df.columns = column_names

In [59]:
# Concatenate the DataFrames vertically (stack them on top of each other)
merged_df = pd.concat(dfs, ignore_index=True)

# Display the merged DataFrame
print(merged_df)



      age sex on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0      41   F            f                  f                         f    f   
1      23   F            f                  f                         f    f   
2      46   M            f                  f                         f    f   
3      70   F            t                  f                         f    f   
4      70   F            f                  f                         f    f   
...    ..  ..          ...                ...                       ...  ...   
11967  56   M            f                  f                         f    f   
11968  22   M            f                  f                         f    f   
11969  69   M            f                  f                         f    f   
11970  47   F            f                  f                         f    f   
11971  31   M            f                  f                         f    f   

      pregnant thyroid_surgery I131_tre

In [61]:
from datetime import datetime

In [65]:
# convert merged file into .csv and save it to artifacts/ingested directory
current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_file_name = f"thyroid_merged_{current_datetime}.csv"
csv_path = os.path.join(ingested_data_directory, csv_file_name)

# Replace '?' with NaN
merged_df.replace('?', pd.NA, inplace=True)
merged_df.to_csv(csv_path, index=False, header=True)